# Trabajo Practico NLP - Detección de Tópicos y clasificación
---


### Codigo de ejemplo simplificado
Les comparto el código básico que yo utilicé, que en gran medida fui adaptandolo de las notebooks de curso.
- Aclaraciones: Es una versión simplificada, por eso se van a utilizar los keywords y entities provistos ( ustedes decidan si luego quieren generar los propios)
- Recomendaciones:
    - Crear un ambiente virtual
    - Dentro de una carpeta del TP colocar esta notebook, TP_tools.py, opensearch_data_model.py, opensearch_io.py     
    - Validar en visual code que estan dentro del ambiente virtual
    - Validar si está levantado opensearch

Recuerden hagan sus propias validaciones del código ( tambien estoy aprendiendo! )

### Librerias necesarias
--> Recuerden instalar con pip install las librerias que no tengan instaladas

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
from tqdm import tqdm
from collections import Counter
from dateutil.parser import parse

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

# -->> levantar la base antes de ejecutar
from opensearch_data_model import Topic, TopicKeyword, os_client, TOPIC_INDEX_NAME
from TP_NLP_tools import init_opensearch, clean_all, Cleaning_text, top_keywords, top_entities, best_document, clean_all, topic_threshold

# -->> necesario para obtener datos de Hugginface
from datasets import load_dataset

### Inicializamos la base vectorial
Se modifica el indice de la base "Topic" agregando referencias del documento mas cercano como el ID y el titulo

In [2]:
# Inicialización de indices
init_opensearch()

El índice Topic ya existe. Saltando inicialización de base de datos.


### Obtenemos los datos de una fecha

In [3]:
# Desde Hugginface https://huggingface.co/jganzabalseenka

date_choice = '2024-07-20'  # formato aaaa-mm-dd ( deben validar en Hugginface que exista data para la fecha que elijan)
path_file = f"jganzabalseenka/news_{date_choice}_24hs"
dataset = load_dataset(path_file)
df_parquet = pd.DataFrame(dataset['train'])
df_parquet.head(1)

,asset_id,title_ch,Asset Destination,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
0,115268180,Esta histórica capital europea fue elegida com...,http://infobae.com/america/mundo/2024/07/20/es...,Infobae,27046,2024-07-20 14:11:36,2024-07-20 11:11:36,[],[],2024-07-20 14:17:48.198436,"[{'entities': [{'end': 30, 'entity_group': 'MI...","[Pilar Alvarez Nuevo, Italki, Edimburgo, Escoc...",Esta histórica capital europea fue elegida com...,"20 Jul, 2024 Por Pilar Alvarez Nuevo Una apli...","[familias jóvenes, modesta verticalidad, mejor...",2024-07-20 14:22:52.533250,"20 Jul, 2024 Por Pilar Alvarez Nuevo Una apli...",Esta histórica capital europea fue elegida com...,0.084537,0.187926


In [4]:
# Cantidad total de documentos
len(df_parquet)

11817

### Validar datos para la fecha
El set de datos de hugginface, a pesar de que es para una fecha determinada, puede traer noticias de dias contigüos.

In [5]:
choice = "".join(date_choice.split('-'))
df_parquet.sort_values("start_time_local", ascending=True, inplace=True)
df_date = df_parquet[df_parquet['start_time_local'].dt.date == pd.to_datetime(choice).date()]
print(f"Registros para la fecha {choice} -> {len(df_date)} de un total de {len(df_parquet)}")

Registros para la fecha 20240720 -> 8008 de un total de 11817


### Obtenemos 1000 noticias por dia (para agilizar el procesamiento)
Mediante sampling

In [6]:
batch_news = 1000

df_1 = df_date.sample(n=int(batch_news)).copy()
df_1.head(1)


,asset_id,title_ch,Asset Destination,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
7818,115274168,Clima en Bariloche: cuál es el pronóstico del ...,http://adnsur.com.ar/sociedad/clima/clima-en-b...,ADN Sur,1109,2024-07-20 16:12:30,2024-07-20 13:12:30,[Servicio Meteorológico Nacional],[Servicio Meteorológico Nacional],2024-07-20 16:13:28.166486,"[{'entities': [{'end': 18, 'entity_group': 'LO...","[Bariloche, Villa La Angostura, Servicio Meteo...",Clima en Bariloche: cuál es el pronóstico del ...,El pronóstico del tiempo para la ciudad de Vil...,"[vientos, velocidades estimadas, el cielo, cli...",2024-07-20 16:21:56.898570,El pronóstico del tiempo para la ciudad de Vil...,Clima en Bariloche: cuál es el pronóstico del ...,0.141314,0.157472


In [7]:
# Obtenemos el texto de las noticias filtradas
data_1 = list(df_1['text'])

# Listas para grabar data en indice topic ( best_doc )
id_data_1 = list(df_1['asset_id'])
title_data_1 = list(df_1['title'])

### StopWords
Dos listas, una base y otra particular para noticias.

In [8]:
# Stopwords
SPANISH_STOPWORDS = list(pd.read_csv('spanish_stop_words.csv' )['stopwords'].values)
SPANISH_STOPWORDS_SPECIAL = list(pd.read_csv('spanish_stop_words_spec.csv' )['stopwords'].values)

#### BOW - Armado del vocabulario con las entidades y keywords

##### Entities

In [9]:
ent_1 = df_1["entities"]
ent_1_set = list(set([ ent.lower() for sublista in ent_1 for ent in sublista ]))
ent_1_clean = clean_all(ent_1_set, accents=False)
ent_1_unique = [ word for word in ent_1_clean if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL]
ent_1_unique[:10]

['iraola',
 'psa',
 'berkshire hathaway',
 'nicolás gaitán',
 'jonás aguirre',
 'gonzalo abrego',
 'decile',
 'grifols',
 'unicaja',
 'carlos pérez']

In [10]:
len(ent_1_unique)

5601

##### Keywords

In [11]:
key_1 = df_1["keywords"]
key_1_set = list(set([ keyw.lower() for sublista in key_1 for keyw in sublista ]))
key_1_clean = clean_all(key_1_set, accents=False)
key_1_unique = [ word for word in key_1_clean if word not in SPANISH_STOPWORDS+SPANISH_STOPWORDS_SPECIAL]
key_1_unique[:10]

['manejo sanitario',
 'encanto',
 'hospitales regionales',
 'mensajes enviados libremente',
 'nivel amarillo',
 'comienzo adverso',
 'villas satélite',
 'bailes sensuales',
 'casa municipal',
 'escritores invitados']

In [12]:
len(key_1_unique)

14395

In [13]:
# Unificar Entities + Keywords 
vocab_1 = list(set(ent_1_unique + key_1_unique))
len(vocab_1)

19526

### Preprocesar las noticias
Se realiza un preprocesamiento mínimo del texto, pero no se le quita el sentido semántico para que mediante SentenceTransformer se puedan capturar embeddings de mejor calidad.

In [14]:
clean_data = Cleaning_text()

proc_data_1 = []
for data_in in tqdm(data_1):
    aux = clean_data.unicode(data_in)
    aux = clean_data.urls(aux)
    aux = clean_data.simbols(aux)
    aux = clean_data.escape_sequence(aux)
    aux = " ".join([ word for word in aux.split() if word.lower() not in SPANISH_STOPWORDS_SPECIAL])
    proc_data_1.append(aux)


100%|██████████| 1000/1000 [00:00<00:00, 2013.77it/s]


### Modelo

In [15]:
tfidf_vectorizer = TfidfVectorizer(
        tokenizer=None,
        max_df=0.9,
        min_df=0.1,
        ngram_range=(1, 3),
        vocabulary=vocab_1,
)
tfidf_vectorizer.fit(data_1)

TfidfVectorizer(max_df=0.9, min_df=0.1, ngram_range=(1, 3),
                vocabulary=['manejo sanitario', 'encanto',
                            'hospitales regionales',
                            'mensajes enviados libremente', 'iraola',
                            'nivel amarillo', 'comienzo adverso', 'psa',
                            'villas satélite', 'bailes sensuales',
                            'berkshire hathaway', 'casa municipal',
                            'escritores invitados', 'actividades', 'extracto',
                            'determinadas legislaciones', 'nicolás gaitán',
                            'jonás aguirre', 'gonzalo abrego', 'balada triste',
                            'decile', 'gestión forestal', 'grifols',
                            'barcos pesqueros', 'unicaja', 'ítem específico',
                            'modelo económico', 'estadio monumental',
                            'almuerzo solidario', 'carrera profesional', ...])

In [16]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
# Step 4 - Tokenize topics
vectorizer_model = tfidf_vectorizer
# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model_1 = BERTopic(
  embedding_model=embedding_model,              # Step 1 - Extract embeddings
  umap_model=umap_model,                        # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,                  # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,            # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                    # Step 5 - Extract topic words
  # language='multilingual',                    # This is not used if embedding_model is used.
  verbose=True,
  # calculate_probabilities=True
)

### Entrenamiento

In [17]:
topics_1, probs_1 = topic_model_1.fit_transform(proc_data_1)

2024-08-18 12:08:45,320 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-18 12:10:31,007 - BERTopic - Embedding - Completed ✓
2024-08-18 12:10:31,007 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-18 12:10:43,082 - BERTopic - Dimensionality - Completed ✓
2024-08-18 12:10:43,096 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-18 12:10:43,115 - BERTopic - Cluster - Completed ✓
2024-08-18 12:10:43,115 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-18 12:10:44,083 - BERTopic - Representation - Completed ✓


### Resultados

In [18]:
# Obtener documentos de cada tópico
topic_freq_1 = topic_model_1.get_topic_freq()

# Imprimir el número de tópicos encontrados (incluyendo el tópico -1)
num_topics_1 = len(topic_freq_1)
print(f"Número de tópicos encontrados: {num_topics_1} (incluye el topico -1)")

# Imprimir la cant de documentos de cada tópico
print(topic_freq_1)

Número de tópicos encontrados: 25 (incluye el topico -1)
    Topic  Count
2      -1    279
10      0    152
11      1     60
1       2     49
19      3     46
18      4     36
7       5     36
0       6     29
17      7     29
12      8     28
6       9     28
8      10     25
14     11     24
4      12     20
5      13     19
24     14     19
9      15     18
13     16     17
15     17     15
3      18     15
21     19     15
22     20     11
16     21     10
20     22     10
23     23     10


In [19]:
### Recuperar todos los topicos y sus etiquetas generadas por el modelo
topic_labels_1 = topic_model_1.generate_topic_labels()
topic_labels_1

['-1_gobierno_com_pensiones',
 '0_liga_liga profesional_la liga',
 '1_poda_santo_sanatorio',
 '2_milei_fmi_biden',
 '3_reservas_banco central_bcra',
 '4_rubinstein_doherty_oxígeno',
 '5_liliana_música_museo',
 '6_temperaturas_vientos_clima',
 '7_policía_sujeto_la policía',
 '8_accidente_rescate_xi',
 '9_oriana_sabatini_paulo',
 '10_amistad_amigos_la luna',
 '11_soto_catalina_crimen',
 '12_peña_loan_maciel',
 '13_tyc sports_sports_mls',
 '14_sociedades_sociedades anónimas_clubes',
 '15_crowdstrike_aso_microsoft',
 '16_azúcar_metas_tauro',
 '17_di maría_benedetto_ángel di maría',
 '18_gaza_isla_la isla',
 '19_blanqueo_ate_upcn',
 '20_siempre juntos_rosario_juntos',
 '21_trelew_chubut_com',
 '22_dólar_temas_dólar solidario',
 '23_ferrante_gladstone_fancy dance']

In [25]:
# Documentos de un topico
topic_id = np.random.randint(len(set(topics_1))-1)
print(f"Topico N°: {topic_id} --> {topic_labels_1[topic_id+1]}" )

docs_per_topics = [i for i, x in enumerate(topic_model_1.topics_) if x == topic_id]
score_docs = topic_model_1.probabilities_[docs_per_topics]

reg_data = []
for i, doc in enumerate(docs_per_topics):
    reg_data.append([df_1.index[doc], doc, df_1.iloc[doc].title, round(score_docs[i],4)])
    

df_query_1 = pd.DataFrame(reg_data, columns=['ID','pos_rel','titulo','score']).sort_values('score', ascending=False, ignore_index=True)
print(len(df_query_1), "docs encontrados")
df_query_1

Topico N°: 7 --> 7_policía_sujeto_la policía
29 docs encontrados


,ID,pos_rel,titulo,score
0,4288,564,Detuvieron a dos personas que intentaron robar...,1.0000
1,748,901,"En libertad condicional, asaltó a una persona ...",1.0000
2,7527,890,Arrojaron al piso a una comerciante y le arreb...,1.0000
3,3394,268,La policía lo fue a identificar en la calle y ...,1.0000
4,2768,592,"Un delincuente de 16 y otro de 20, detenidos e...",1.0000
5,11059,317,La apuñalaron para robarle y no registraron su...,1.0000
6,11723,389,Detallaron cómo fue el nuevo golpe al negocio ...,1.0000
7,3488,470,Dos mujeres fueron aprehendidas por robo por l...,1.0000
8,7940,126,Gualeguay: Dos policías fueron imputados por robo,1.0000
9,1795,542,Dos detenidos por robar mercadería en supermer...,1.0000


In [26]:
# Obtenemos embeddings de todos los documentos
docs_embedding_1 = topic_model_1.embedding_model.embed(data_1)

In [27]:
topic_model_1.generate_topic_labels()[1]

'0_liga_liga profesional_la liga'

### Grabar topicos en el indice

In [28]:
# Grabar todos los topicos en la base
for topic in topic_model_1.get_topics().keys():
    if topic > -1:

        topic_keywords_top  = top_keywords(topic, topic_model_1)
        topic_entities_top  = top_entities(topic, topic_model_1, df_1)
        threshold  = topic_threshold(topic, topic_model_1, probs_1)
        id_best_doc, title_best_doc, best_doc = best_document(topic, topic_model_1, docs_embedding_1, id_data_1, title_data_1, data_1)

        topic_doc = Topic(
            index = topic,
            name = topic_model_1.generate_topic_labels()[topic + 1],
            vector = list(topic_model_1.topic_embeddings_[topic + 1 ]),
            similarity_threshold = threshold,
            created_at = datetime.now(),
            from_date = parse(date_choice),
            to_date = datetime.strptime(date_choice, '%Y-%m-%d') + timedelta(days=1),
            keywords = topic_keywords_top,
            entities = topic_entities_top,
            id_best_doc = id_best_doc,
            title_best_doc = title_best_doc,
            best_doc = best_doc,
        )

        topic_doc.save()

### Merge de modelos

Para la fusion de modelos:
- repetir el proceso anterior ( pero sin grabar topicos en el indice )
- fusionar modelos para luego obtener los nuevos topicos y actualizar en el indice de opensearch

In [ ]:
# merged_model = BERTopic.merge_models([topic_model_1, topic_model_2])